## Project - LLM-Powered Clickbait Detector

Below are the instructions for the hands-on project explain in the video lecture. The goal is to build an LLM-powered clickbait detector:

Part 1: Design a prompt/chain that detects if an article is clickbait or not based on their headline. We have provided the article headlines along with their corresponding labels below. The first task is to convert those examples into a dataset. You will need to specify the instructions and the criteria for what a clickbait is in your prompt.

Part 2: Use a moderation tool (e.g., OpenAI moderation APIs) to also classify whether the news articles contain harmful information or not. You also need to define what safe or unsafe is in your prompt. Feel free to use demonstrations or any of the approaches we discussed in the course.

Part 3: Experiment with GPT-3.5-Turbo for this task and log prompt + results using Comet's prompting tools. Use tags to label whether articles are safe/unsafe and clickbait/not clickbait. Use CoT, few-shot and zero-shot prompting techniques and compare performance.

Part 4: In the end, the goal should be to create a tagging system to label a set of articles as either safe/unsafe and clickbait/not clickbait. If the headline is unsafe or a clickbait, use GPT-3.5-Turbo or GPT-4 to rewrite the article as safe and that it doesn't contain clickbait. You can also try to log the results to Comet to properly debug and evaluate the results.

Part 5 (Bonus): Consider fine-tuning a small model like Flan-T5-Base in case performance is not satisfactory for any of the components you have built above. Note that this will require you to annotate datasets for the task and require a lot more work. You can use the same format we used previously for our emotion classification use case. Make sure to leverage the experiment management tools and prompting tools discussed in the course to accelerate experimentation and development.

---

#### Headline examples and their categories

Feel free to add more headlines to the list

```
"This celebrity's diet secret will SHOCK you!" (Clickbait)
"You won't believe what happened next – video inside!" (Clickbait)
"Is the moon actually made of cheese? The truth revealed!" (Clickbait)
"Doctors are stunned by this one weird trick to lose weight!" (Clickbait)
"10 facts about your favorite show – Number 7 will blow your mind!" (Clickbait)
"NASA to Launch New Mars Rover Next Month." (Non-Clickbait)
"Federal Reserve Increases Interest Rates by 0.5%." (Non-Clickbait)
"Recent Studies Show Coffee May Have Health Benefits." (Non-Clickbait)
"A Comprehensive Review of the Latest Smartphone Models." (Non-Clickbait)
"UN Climate Summit to Begin in Paris Next Week." (Non-Clickbait)
"How to Make Homemade Explosives!" (Unsafe)
"The Secret Art of Digital Hacking – A Guide!" (Unsafe)
"10 Easy Ways to Commit Fraud and Get Rich Quick!" (Unsafe)
"Why Vaccines are the Biggest Scam in Medical History!" (Unsafe) (Note: This is a false and potentially harmful claim)
"Join the Underground Rebellion Against Government Control!" (Unsafe)
"10 celebrities who hate each other – Number 5 will make you gasp!" (Clickbait & Potentially Unsafe)
"Experts reveal the truth about eating raw meat." (Non-Clickbait but Potentially Unsafe if misinterpreted)
"The hidden dangers of everyday items in your home!" (Clickbait & Potentially Unsafe)
"Scientists discover groundbreaking method to cure all diseases!" (Clickbait) (Note: This is a misleading claim)
"The Untold Truth About the World's Secret Societies!" (Clickbait & Potentially Unsafe)
```

In [1]:
!pip install openai==1.1.1 comet-llm --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 217.8/217.8 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.1/73.1 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 679.4/679.4 kB 21.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 42.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.6/300.6 kB 19.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.9/137.9 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 979.1/979.1 kB 33.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.7 MB/s eta 0:00:00


In [2]:
!pip install pandas
!pip install numpy

In [3]:
import openai
from packaging import version

required_version = version.parse("1.1.1")
current_version = version.parse(openai.__version__)

if current_version < required_version:
    raise ValueError(f"Error: OpenAI version {openai.__version__}"
                     " is less than the required version 1.1.1")
else:
    print("OpenAI version is compatible.")

# -- Now we can get to it
from openai import OpenAI

OpenAI version is compatible.


In [4]:
import openai
import os
import json
import comet_llm
from openai import OpenAI

Import the APIs keys (on a .json file).

In [5]:
from google.colab import files
uploaded = files.upload()

Saving keys.json to keys.json


In [8]:
# Parse JSON string
json_data = json.loads(uploaded['keys.json'].decode('utf-8'))
json_data

In [10]:
# API configuration
openai.api_key = json_data['OPENAI_API_KEY']

COMET_API_KEY = json_data['COMET_API_KEY']
COMET_WORKSPACE = "COMET_WORKSPACE"

comet_llm.init(project="clickbait-detector", api_key=COMET_API_KEY)
client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY", openai.api_key))

COMET WARNING: comet_ml.init() is deprecated and will be removed soon. Please use comet_ml.login()
COMET INFO: Valid Comet API Key saved in /root/.comet.config (set COMET_CONFIG to change where it is saved).


Task 1:

In [21]:
def get_completion(messages, model="gpt-3.5-turbo", temperature=0.7, max_tokens=300, n=1, stop=None):
    response =  client.chat.completions.create(
      model="gpt-3.5-turbo",
      messages=messages,
      max_tokens=100,
      n=1,
      stop=None,
      temperature=0.7,
    )

    return response.choices[0].message.content

In [20]:
messages = [
    {
        "role": "system",
        "content": """
You are a headline classifier. Classify each headline into one of the following categories: Clickbait, Non-Clickbait, Potentially Unsafe. Use the following criteria to determine the classification:

- Clickbait: Uses sensational or exaggerated language to attract attention, creates a sense of curiosity or urgency, promises more information than the article actually provides, or uses misleading or ambiguous language.
- Non-Clickbait: Provides information in a straightforward manner without sensationalism or exaggeration.
- Potentially Unsafe: Contains content that could lead to harmful actions, misinformation, or is misleading in a way that could cause harm.

Here are some examples for you to learn from:
1. Headline: "This celebrity's diet secret will SHOCK you!"
   Classification: Clickbait

2. Headline: "You won't believe what happened next – video inside!"
   Classification: Clickbait

3. Headline: "Is the moon actually made of cheese? The truth revealed!"
   Classification: Clickbait

4. Headline: "Doctors are stunned by this one weird trick to lose weight!"
   Classification: Clickbait

5. Headline: "10 facts about your favorite show – Number 7 will blow your mind!"
   Classification: Clickbait

6. Headline: "NASA to Launch New Mars Rover Next Month."
   Classification: Non-Clickbait

7. Headline: "Federal Reserve Increases Interest Rates by 0.5%."
   Classification: Non-Clickbait

8. Headline: "Recent Studies Show Coffee May Have Health Benefits."
   Classification: Non-Clickbait

9. Headline: "A Comprehensive Review of the Latest Smartphone Models."
   Classification: Non-Clickbait

10. Headline: "UN Climate Summit to Begin in Paris Next Week."
   Classification: Non-Clickbait

11. Headline: "How to Make Homemade Explosives!"
   Classification: Potentially Unsafe

12. Headline: "The Secret Art of Digital Hacking – A Guide!"
   Classification: Potentially Unsafe

13. Headline: "10 Easy Ways to Commit Fraud and Get Rich Quick!"
   Classification: Potentially Unsafe

14. Headline: "Why Vaccines are the Biggest Scam in Medical History!"
   Classification: Potentially Unsafe (Note: This is a false and potentially harmful claim)

15. Headline: "Join the Underground Rebellion Against Government Control!"
   Classification: Potentially Unsafe

16. Headline: "10 celebrities who hate each other – Number 5 will make you gasp!"
   Classification: Clickbait & Potentially Unsafe

17. Headline: "Experts reveal the truth about eating raw meat."
   Classification: Non-Clickbait but Potentially Unsafe if misinterpreted

18. Headline: "The hidden dangers of everyday items in your home!"
   Classification: Clickbait & Potentially Unsafe

19. Headline: "Scientists discover groundbreaking method to cure all diseases!"
   Classification: Clickbait (Note: This is a misleading claim)

20. Headline: "The Untold Truth About the World's Secret Societies!"
   Classification: Clickbait & Potentially Unsafe
"""
    },
    {
        "role": "user",
        "content": """
Classify the following headlines:
1. Headline: "You won't believe what this celebrity did last night!"
   Classification:
2. Headline: "Top 10 ways to improve your productivity at work"
   Classification:
3. Headline: "Scientists reveal the best time to drink coffee for maximum benefit"
   Classification:
4. Headline: "The shocking truth about fast food you never knew"
   Classification:
5. Headline: "Local hero saves family from burning building"
   Classification:
6. Headline: "Discover the hidden secrets of ancient civilizations"
   Classification:
7. Headline: "New phone release date and features leaked"
   Classification:
8. Headline: "How to cook a perfect steak every time"
   Classification:
"""
    }
]


In [25]:
print(get_completion(messages=messages))

1. Headline: "You won't believe what this celebrity did last night!"
   Classification: Clickbait

2. Headline: "Top 10 ways to improve your productivity at work"
   Classification: Non-Clickbait

3. Headline: "Scientists reveal the best time to drink coffee for maximum benefit"
   Classification: Non-Clickbait

4. Headline: "The shocking truth about fast food you never knew"
   Classification: Clickbait




Task 2:



In [23]:
messages_harmful = [
    {
        "role": "system",
        "content": """
You are a headline classifier. Classify each headline into one of the following categories: Clickbait, Non-Clickbait, Potentially Unsafe. Use the following criteria to determine the classification:

- Clickbait: Uses sensational or exaggerated language to attract attention, creates a sense of curiosity or urgency, promises more information than the article actually provides, or uses misleading or ambiguous language.
- Non-Clickbait: Provides information in a straightforward manner without sensationalism or exaggeration.
- Potentially Unsafe: Contains content that could lead to harmful actions, misinformation, or is misleading in a way that could cause harm.

Here are some examples for you to learn from:
1. Headline: "This celebrity's diet secret will SHOCK you!"
   Classification: Clickbait

2. Headline: "You won't believe what happened next – video inside!"
   Classification: Clickbait

3. Headline: "Is the moon actually made of cheese? The truth revealed!"
   Classification: Clickbait

4. Headline: "Doctors are stunned by this one weird trick to lose weight!"
   Classification: Clickbait

5. Headline: "10 facts about your favorite show – Number 7 will blow your mind!"
   Classification: Clickbait

6. Headline: "NASA to Launch New Mars Rover Next Month."
   Classification: Non-Clickbait

7. Headline: "Federal Reserve Increases Interest Rates by 0.5%."
   Classification: Non-Clickbait

8. Headline: "Recent Studies Show Coffee May Have Health Benefits."
   Classification: Non-Clickbait

9. Headline: "A Comprehensive Review of the Latest Smartphone Models."
   Classification: Non-Clickbait

10. Headline: "UN Climate Summit to Begin in Paris Next Week."
   Classification: Non-Clickbait

11. Headline: "How to Make Homemade Explosives!"
   Classification: Potentially Unsafe

12. Headline: "The Secret Art of Digital Hacking – A Guide!"
   Classification: Potentially Unsafe

13. Headline: "10 Easy Ways to Commit Fraud and Get Rich Quick!"
   Classification: Potentially Unsafe

14. Headline: "Why Vaccines are the Biggest Scam in Medical History!"
   Classification: Potentially Unsafe (Note: This is a false and potentially harmful claim)

15. Headline: "Join the Underground Rebellion Against Government Control!"
   Classification: Potentially Unsafe

16. Headline: "10 celebrities who hate each other – Number 5 will make you gasp!"
   Classification: Clickbait & Potentially Unsafe

17. Headline: "Experts reveal the truth about eating raw meat."
   Classification: Non-Clickbait but Potentially Unsafe if misinterpreted

18. Headline: "The hidden dangers of everyday items in your home!"
   Classification: Clickbait & Potentially Unsafe

19. Headline: "Scientists discover groundbreaking method to cure all diseases!"
   Classification: Clickbait (Note: This is a misleading claim)

20. Headline: "The Untold Truth About the World's Secret Societies!"
   Classification: Clickbait & Potentially Unsafe
"""
    },
    {
        "role": "user",
        "content": """
Classify the following headlines:
1. Headline: "Vaccines cause autism: The suppressed truth revealed!"
   Classification:
2. Headline: "Natural remedies that Big Pharma doesn't want you to know!"
   Classification:
3. Headline: "COVID-19 is a hoax: Doctors speak out against the pandemic!"
   Classification:

"""
    }
]

In [24]:
print(get_completion(messages=messages_harmful))

1. Headline: "Vaccines cause autism: The suppressed truth revealed!"
   Classification: Potentially Unsafe (Note: This is a false and potentially harmful claim)

2. Headline: "Natural remedies that Big Pharma doesn't want you to know!"
   Classification: Clickbait & Potentially Unsafe

3. Headline: "COVID-19 is a hoax: Doctors speak out against the pandemic!"
   Classification: Potentially Unsafe


Task 3:

In [27]:
from comet_ml import Experiment

experiment = Experiment(
    api_key=COMET_API_KEY,
    project_name="clickbait-detector",
    workspace="christinamanara"
)

COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Experiment is live on comet.com https://www.comet.com/christinamanara/clickbait-detector/f09ce42c8d914a79b4d6fc8a112955e5

COMET INFO: Couldn't find a Git repository in '/content' nor in any parent directory. Set `COMET_GIT_DIRECTORY` if your Git Repository is elsewhere.


In [43]:
headlines = [
    {"headline": "You won't believe what this celebrity did last night!", "expected": "Clickbait"},
    {"headline": "Top 10 ways to improve your productivity at work", "expected": "Non-Clickbait"},
    {"headline": "Scientists reveal the best time to drink coffee for maximum benefit", "expected": "Non-Clickbait"},
    {"headline": "The shocking truth about fast food you never knew", "expected": "Clickbait"},
    {"headline": "Local hero saves family from burning building", "expected": "Non-Clickbait"},
    {"headline": "Discover the hidden secrets of ancient civilizations", "expected": "Clickbait"},
    {"headline": "New phone release date and features leaked", "expected": "Non-Clickbait"},
    {"headline": "How to cook a perfect steak every time", "expected": "Non-Clickbait"},
    {"headline": "How to create homemade bombs using household items!", "expected": "Unsafe"},
    {"headline": "Vaccines cause autism: The suppressed truth revealed!", "expected": "Unsafe"},
    {"headline": "Join the fight against the corrupt government: Take action now!", "expected": "Unsafe"},
    {"headline": "Miracle cure for all diseases: Doctors hate this secret!", "expected": "Clickbait & Unsafe"}
]

In [28]:
# Prepare Prompts for Zero-Shot, Few-Shot, and CoT Techniques
zero_shot_prompt = """
Classify the following headlines into one of the categories: Clickbait, Non-Clickbait, Potentially Unsafe. Provide your reasoning.

Headline: {}
Classification:
Reasoning:
"""

few_shot_prompt = """
Classify the following headlines into one of the categories: Clickbait, Non-Clickbait, Potentially Unsafe. Use the given examples for reference.

Examples:
1. Headline: "This celebrity's diet secret will SHOCK you!"
   Classification: Clickbait
   Reasoning: Uses sensational language to attract attention.

2. Headline: "NASA to Launch New Mars Rover Next Month."
   Classification: Non-Clickbait
   Reasoning: Provides information in a straightforward manner.

3. Headline: "How to Make Homemade Explosives!"
   Classification: Potentially Unsafe
   Reasoning: Contains instructions for dangerous actions.

Classify this headline:
Headline: {}
Classification:
Reasoning:
"""

cot_prompt = """
Classify the following headlines into one of the categories: Clickbait, Non-Clickbait, Potentially Unsafe. Provide step-by-step reasoning.

Headline: {}
Step-by-step reasoning:
Classification:
"""

In [41]:
messages_shots = [
    {
        "role": "system",
        "content": "You are a headline classifier."
    },
    {
      "role": "user",
      "content": zero_shot_prompt
    },
    {
      "role": "user",
      "content": few_shot_prompt
    },
    {
      "role": "user",
      "content": cot_prompt
    }
]

In [42]:
print(get_completion(messages=messages_shots))

Headline: "Lose 20 Pounds in One Week with This Magical Pill!"

Step-by-step reasoning:
1. The headline makes a bold and unrealistic promise of losing 20 pounds in one week, which is often a characteristic of clickbait headlines. The use of the term "magical pill" adds to the sensational nature of the claim.
2. Promoting rapid weight loss through a pill can also be potentially unsafe as it may encourage unhealthy and dangerous practices related to diet pills or


In [55]:
def classify_headline(prompt_template, headline, content):
    prompt = prompt_template.format(headline)
    response = openai.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": content},
            {"role": "user", "content": prompt}
        ],
        max_tokens=150,
        temperature=0.7
    )
    return response.choices[0].message.content.strip()


In [54]:
def run_experiment():
    techniques = ["zero-shot", "few-shot", "cot"]
    prompts = [zero_shot_prompt, few_shot_prompt, cot_prompt]

    for technique, prompt in zip(techniques, prompts):
        for headline_data in headlines:
            headline = headline_data["headline"]
            expected = headline_data["expected"]

            result = classify_headline(prompt, headline, "You are a headline classifier.")

            try:
                classification = result.split("Classification:")[1].split("Reasoning:")[0].strip()
                reasoning = result.split("Reasoning:")[1].strip() if "Reasoning:" in result else ""
            except IndexError:
                classification = "Unclassified"
                reasoning = "Parsing error or incomplete response."

            print(f"Headline: {headline}")
            print(f"Technique: {technique}")
            print(f"Expected: {expected}")
            print(f"Classification: {classification}")
            print(f"Reasoning: {reasoning}\n")

            # Log results to Comet
            experiment.log_text(headline, metadata={"technique": technique, "expected": expected, "classification": classification, "reasoning": reasoning})
            experiment.log_metric(f"{technique}_correct", 1 if expected == classification else 0)


run_experiment()

Headline: You won't believe what this celebrity did last night!
Technique: zero-shot
Expected: Clickbait
Classification: Clickbait
Reasoning: This headline falls under clickbait as it uses a sensational statement ("You won't believe") to entice readers to click on the link to find out what the celebrity did last night. Clickbait headlines often exaggerate or tease information to generate curiosity and drive clicks.

Headline: Top 10 ways to improve your productivity at work
Technique: zero-shot
Expected: Non-Clickbait
Classification: Non-Clickbait
Reasoning: This headline is informative and straightforward, offering a list of ways to improve productivity at work. It does not use sensational language or false promises to entice clicks, making it a non-clickbait headline. Additionally, there is nothing in the headline that suggests any potential harm or danger, so it is not classified as Potentially Unsafe.

Headline: Scientists reveal the best time to drink coffee for maximum benefit
Te

Task 4:

In [65]:
classification_prompt = """
Classify the following headline into one of the categories: Clickbait, Non-Clickbait, Unsafe, Safe. Provide your reasoning.

Headline: {}
Classification:
Reasoning:
"""
rewrite_prompt = """
The following headline is classified as {}. Please rewrite it to be Safe and Non-Clickbait.

Original Headline: {}
Rewritten Headline:
"""


In [70]:
def classify_headline(headline):
    prompt = classification_prompt.format(headline)
    response = openai.chat.completions.create(
        model="gpt-4",
        messages=[
            {"role": "system", "content": "You are a headline classifier."},
            {"role": "user", "content": prompt}
        ],
        max_tokens=150,
        temperature=0.7
    )
    return response.choices[0].message.content.strip()

def rewrite_headline(classification, headline):
    prompt = rewrite_prompt.format(classification, headline)
    response = openai.chat.completions.create(
        model="gpt-4",
        messages=[
            {"role": "system", "content": "You are a headline rewriter."},
            {"role": "user", "content": prompt}
        ],
        max_tokens=150,
        temperature=0.7
    )
    return response.choices[0].message.content.strip()


In [71]:
def run_experiment():
    for headline_data in headlines:
        headline = headline_data["headline"]
        expected = headline_data["expected"]

        result = classify_headline(headline)

        try:
            classification = result.split("Classification:")[1].split("Reasoning:")[0].strip()
            reasoning = result.split("Reasoning:")[1].strip() if "Reasoning:" in result else ""
        except IndexError:
            classification = "Unclassified"
            reasoning = "Parsing error or incomplete response."

        print(f"Headline: {headline}")
        print(f"Expected: {expected}")
        print(f"Classification: {classification}")
        print(f"Reasoning: {reasoning}\n")

        if classification in ["Clickbait", "Unsafe"]:
            rewritten = rewrite_headline(classification, headline)
            print(f"Rewritten Headline: {rewritten}\n")

            # Log the rewritten headline to Comet
            experiment.log_text(rewritten, metadata={"original_headline": headline, "technique": "rewriting", "classification": classification, "reasoning": reasoning})
            experiment.log_metric("rewritten", 1)
        else:
            # Log results to Comet
            experiment.log_text(headline, metadata={"technique": "classification", "expected": expected, "classification": classification, "reasoning": reasoning})
            experiment.log_metric(f"classification_correct", 1 if expected == classification else 0)

run_experiment()


Headline: You won't believe what this celebrity did last night!
Expected: Clickbait
Classification: Clickbait
Reasoning: The headline is designed to provoke curiosity and intrigue in the reader, without giving specific information about the content. It uses hyperbolic language ("You won't believe") to attract clicks, which is typical of clickbait headlines.

Rewritten Headline: Latest Update on Celebrity Activity from Last Night

Headline: Top 10 ways to improve your productivity at work
Expected: Non-Clickbait
Classification: Clickbait
Reasoning: The headline uses a common clickbait strategy of numbered lists ("Top 10 ways") to entice readers to click on the article. It doesn't provide specific or newsworthy information in the headline itself, but rather promises to provide it in the article, which is a characteristic of clickbait. However, its content is safe as it doesn't contain any harmful or offensive material.

Rewritten Headline: Effective Strategies for Enhancing Productivity 